In [10]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import re
import time
import requests
from bs4 import BeautifulSoup
import PyPDF2
import pandas as pd
import os
import io

In [4]:
def get_doi_from_ieee_page(url):
    # Set up the Firefox WebDriver options
    firefox_options = Options()
    firefox_options.add_argument("--headless")  # Run in headless mode (no GUI)
    gecko_driver_path =os.getcwd()+'/geckodriver/geckodriver.exe'
    # Specify the path to the Firefox binary if it's in a non-standard location
    firefox_options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # <-- Update this to your Firefox path
    service = Service(gecko_driver_path)
    # Use webdriver-manager to automatically download and manage GeckoDriver
    driver = webdriver.Firefox(service=service, options=firefox_options)

    
    try:
        # Navigate to the IEEE Xplore paper page URL
        driver.get(url)
        
        # Wait for the page to load fully
        time.sleep(5)  # Adjust based on your connection speed
        
        # Get the page source after JavaScript execution
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # Find the DOI on the page (DOIs are typically found in a meta tag or a specific link)
        
        text = soup.prettify()
        pattern = r'doi.org/(.*?)(?=")'
        match = re.search(pattern, text)
        doi = match.group(1)
        return doi
    except Exception as e:
        print(Exception)
    finally:
        # Close the browser after scraping
        driver.quit()
def download_pdf(sci_hub_url):
    # Fetch the Sci-Hub page
    response = requests.get(sci_hub_url)
    
    # Parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the download button and extract the URL from its onclick attribute
    pdf_link = None
    pdf_link = None
    button = soup.find('button', onclick=re.compile(r'location\.href='))

    if button:
        # Extract the link from the onclick attribute
        onclick_value = button['onclick']
        match = re.search(r"location\.href='(.*?)'", onclick_value)
        if match:
            pdf_link = match.group(1)

    if pdf_link is None:
        print("PDF link not found.")
        return None

    # Make the PDF link absolute
    pdf_link = requests.compat.urljoin(sci_hub_url, pdf_link)

    # Download the PDF
    pdf_response = requests.get(pdf_link)
    return pdf_response.content

def read_pdf(pdf_content):
    # Read the PDF content
    pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_content))
    full_text = ''
    
    for page in pdf_reader.pages:
        full_text += page.extract_text() + '\n'
    
    return full_text

### One Link

In [11]:
url = "https://ieeexplore.ieee.org/document/9659697"
doi = get_doi_from_ieee_page(url)
sci_link = 'https://sci-hub.se/'+doi
pdf_content = download_pdf(sci_link)


PDF link not found.


### Production

In [6]:
path = '../Scrapes_ALL.csv'
df = pd.read_csv(path)
errors = []
all_links = [i for i in list(df['URL']) if 'ieee' in i]
full_texts = []
print(len(all_links))
for i in all_links:
    try:
        url = i
        print(url)
        doi = get_doi_from_ieee_page(url)
        sci_link = 'https://sci-hub.se/'+doi
        pdf_content = download_pdf(sci_link)
        print(sci_link)
        if pdf_content:
            full_text = read_pdf(pdf_content)
            full_texts.append(full_text)
        else:
            full_texts.append('Nothing found')
    except:
        full_texts.append('Error')
        errors.append(i)

FileNotFoundError: [Errno 2] No such file or directory: 'Scrapes_ALL.csv'

In [12]:
full_df = pd.DataFrame({'links':all_links, 'texts':full_texts})
path = '../results/ieee_full_texts.csv'
full_df.to_csv(path, index=False)

In [7]:
len(full_texts)

76

In [13]:
full_df

,links,texts
0,https://ieeexplore.ieee.org/document/4563052,3D Facial Expression Recognition Based on Auto...
1,https://ieeexplore.ieee.org/document/1640921,3D Facial Expression Recognition Based on Prim...
2,https://ieeexplore.ieee.org/document/4813304,3DFacial Expr ession Recognition Based onPrope...
3,https://ieeexplore.ieee.org/document/1613022,A 3D Facial Expression Database For Facial Be...
4,https://ieeexplore.ieee.org/document/5975141,A Multimodal Database for\nAffect Recognition ...
...,...,...
71,https://ieeexplore.ieee.org/document/6130508,Nothing found
72,https://ieeexplore.ieee.org/document/5543262,Nothing found
73,https://ieeexplore.ieee.org/document/5771374,Nothing found
74,https://ieeexplore.ieee.org/document/4459336,Nothing found


In [39]:
counter = 0
for i in full_texts:
    if i == 'Error' or i == 'Nothing found':
        counter+=1
print(counter)

11
